## 4. <a id='4_cell'></a>Create Group using existing Da Vinci Alerts Data (from Synthea) and the PyFHIR models to...

- create Group resources for discovery by:

    1. all patients
    1. all patients with managing org
    1. practitioner characteristic
    1. location characteristic
    *TODO*
    - *add narrative*
    - *add Careteams and Careteam characteristics*
- Create the Q/QR data using existing tools and add extensions to resources
- Create transaction bundle of resources
- Load to ref server
- Fetching and...
- Getting Additional data by:
    1. FHIR RESTful queries on the:
       - Patient
       - *TODO Condition and*
       - Encounter resources
      

### Create Group Instance for All Patients, One member for Each Row...

1. initialize group resource and datatype instances
1. iterate over each Patient in Bundle
1. validate and save the resources in the reference server
    tag them so can search more easily

TODO
1. option to create narrative using jinja2 templates (that is a topic for another notebook)
1. create a new transaction Bundle with all the data (figure out sequence for encounters)

### import all the modules you need

In [7]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# This allows you to import the desired function from the module hierarchy:
from pyFHIR_models.fhir_model_generator.model import (
                                            fhirdate,
                                            fhirreference,
                                            group,
                                            patient,
                                            organization,
                                            bundle,
                                            identifier,
                                            humanname,
                                            contactpoint,
                                            address,
                                            codeableconcept,
                                            coding,
                                            extension,
                                            questionnaireresponse as qr,
                                             )
from json import loads, dumps
#from requests import get, post, put
from datetime import datetime, date, timedelta
from IPython.display import display as Display, HTML, Markdown
from utils import bundle_me, to_json, to_yaml, get_id, load_transaction, validate, fetch_r
from pathlib import Path
#from pprint import pprint, pformat
from nested_lookup import nested_lookup
import FHIR_templates

## Globals

file location etc...

'pl_use_case':

|value|what it does|
|---|---|
|all| create Group with no characterisitics|
|location| create Group with location characteristic|
|practitioner| create Group with practitioner characteristic|


In [8]:
"""
group_characterstics = [
(0,'location','Location/[id]',),
(1,'practitioner','Practitioner/[id]',),
(2,'organization','Organization/[id]',),
(3,'team','CareTeam/[id]',),
]
"""

base_path = '/Users/ehaas/Documents/FHIR/Davinci-Alerts/2020_09_hl7_connectathon/Synthea_Alert_Test_Data'

file_type = 'fhir'
#file_type = 'csv'

#file_size = '3Patients'
file_size = '100Patients'

#file = 'discharge_notify-2.json'
#file = 'admit_notify-100.json'
file = 'admit_notify-100r1.json'
#file = 'admit_notify-100-BEVERLY HOSPITAL CORPORATION.json'
#file = 'admit_notify-100-HALLMARK HEALTH SYSTEM.json'
#file = 'admit_notify-100-CAMBRIDGE HEALTH ALLIANCE.json'


mypath = Path() / base_path / file_type / file_size
my_infile = mypath/file

print(f'My Path to Synthea Data a FHIR Bundle is {my_infile}')
print()

console_only = False

chars = dict(  # this is dict to provide the FHIR path to the reference in Encounters to get data
#all = None,
location = 'location.location',
practitioner = 'participant.individual',
)

example_series = dict(
all = 100,
location = 200,
practitioner = 300,
)

#pl_use_case = "location"
#pl_use_case = "practitioner"
pl_use_case = "all"

char_system = 'http://argonautproject.org/patient-lists/CodeSystem/group-characteristic'

q_ext_url = 'http://registry.fhir.org/argonaut/StructureDefinition/patientlistsquestionnaire'
qr_ext_url = 'http://registry.fhir.org/argonaut/StructureDefinition/patientlistquestionnaireresponse'
q_id = 'argo-pl-q1'

display(HTML(f'<h1>USE CASE = {pl_use_case}</h1>'))

My Path to Synthea Data a FHIR Bundle is /Users/ehaas/Documents/FHIR/Davinci-Alerts/2020_09_hl7_connectathon/Synthea_Alert_Test_Data/fhir/100Patients/admit_notify-100r1.json



## Functions

In [9]:
def get_my_chars():  #  get reference based on dot notation string
    my_chars = []
    print(f'get ref from Encounter.{chars[pl_use_case]}')
    for i in my_bundle.entry:
        r = i.resource
        if r.resource_type == "Encounter" and get_id(r.serviceProvider) == get_id(me):
            #print(f'r.serviceProvider.reference == {get_id(r.serviceProvider)} /n me.reference = {get_id(me)}')
            e = r
            
            for j in chars[pl_use_case].split("."):  # creat a util for this nested getattr
                #print(j)
                try:
                    e = getattr(e,j)
                except AttributeError:  # maybe a list so try getting the first element of list
                    e = getattr(e[0],j)
            #print(e)
            my_chars.append(e)
    #print('my-chars', len(my_chars))
    my_chars = dedup(my_chars)
    #print('my-chars deduped',len(my_chars))
    return my_chars
              
    
def add_char(): #Add A Single Group Characteristic
    char.reference = f'{pl_use_case.capitalize()}/{get_id(char)}'
    #print(to_yaml(me))
    my_g.characteristic = []
    my_char = group.GroupCharacteristic()
    my_char.code = codeableconcept.CodeableConcept()
    my_char.code.coding = [coding.Coding()]
    my_char.code.coding[0].code = pl_use_case
    my_char.code.coding[0].display = pl_use_case.capitalize()
    my_char.code.coding[0].system = char_system
    my_char.valueReference = char
    my_char.exclude = False
    my_g.characteristic.append(my_char)
    
    

def dedup(my_refs):
    deduped = [fhirreference.FHIRReference(dict(t)) for t in {tuple(d.as_json().items()) for d in my_refs}] #dedup
    return deduped
    
    

def add_qr():  # Add the QR
    pass
    
def counter():
     if file_size == '3Patients':
        return len(my_groups) + example_series[pl_use_case]
     else:
        return len(my_groups) + example_series[pl_use_case]*10
    
def create_new_group(counter):
    my_id = f'argo-pl-group-{pl_use_case}-{counter}'
    my_meta = dict(
        source = "Health_eData_Inc",
        tag = [{'code': '2020-Sep'}],
        lastUpdated= f'{datetime.now().isoformat()}Z'
        )

    my_identifier = dict(
            value = my_id,
            system = "http://example.org/fhir/identifiers",
            )
    
    q_ref = dict(
    reference = f'Questionnaire/{q_id}',
    display = f'Argo PL List Test Questionnaire id={q_id}',
     )
    
    my_q_extension = dict(
     url = q_ext_url,
     valueReference = q_ref,
    )

    my_g = group.Group(
            dict(
                meta = my_meta,
                id = my_id,
                extension = [my_q_extension],
                type = 'person',
                actual = True,
                name = f'Argo Patient List #{counter}: "{pl_use_case}" Use Case',
                active = True,
                identifier = [my_identifier],
                #quantity = df.shape[0],         
            )
    ) # initialize group instance
    
    # Add the managingEntity
    me.reference = f'Organization/{get_id(me)}'
    #print(to_yaml(me))
    my_g.managingEntity = me

    #print(to_yaml(my_g))
    return my_g


### Add managingEntity element, A Single Group Characteristic and Q extensions

Fetch the FHIR Bundle of Synthea Data -
- path variables are local for this - change as needed

#### add the managingEntity element - i.e. the Organization that created the group.
 - note there are no managing Org on these patients. so for this data set we impute the organization from the the encounter to tell us which organization the patient list go with.
   (This is an issue that is system dependent only they know)
- use nested lookup to extract the Me from encounter
- use FHIR references for now ( may need to switch back to urns for bundling)

#### Add A Single Group Characteristic

- based on Encounter add Location and Practitioner to start 
- TODO add Organization and CareTeam when define these resources ( what does Organization mean?)
- create a function and run through the parameters to build the element

#### Add A Q Extensions

- Add extension to all Groups first
- Create and use fixed templates Q's from NIH Tool
- See if tooling can autopopulate QRs otherwise use my own base on FHIRpath


In [10]:
'''
json_bundles = []
for f in mypath.iterdir():
    #print(type(f), f)
    json_bundles.append(f.read_text(encoding='utf-8'))
'''

my_groups = []
my_bundle = bundle.Bundle(loads(my_infile.read_text(encoding='utf-8')))
print(f'len(my_bundle.entry)={len(my_bundle.entry)}' )
my_refs = [r.resource.serviceProvider for r in my_bundle.entry if r.resource.resource_type == "Encounter"]
#my_refs = my_refs['serviceProvider']
my_refs = dedup(my_refs)
for i in my_refs:
    display(HTML('<h1>MY_REFS AS YAML</h1>'))
    display(HTML(f'<pre style="color: blue; background-color: AliceBlue;">{to_yaml(i)}</pre>'))

len(my_bundle.entry)=561


In [11]:
my_groups = []

for me in my_refs:
    
    if pl_use_case in chars.keys():
        display(HTML(f'<h3>Use Case = {pl_use_case} for {me.display}</h3>'))
        my_chars = get_my_chars() # get all chars for me 
        #print(my_chars)
        for char in my_chars:
            my_g = create_new_group(counter())
            add_char()
            my_groups.append(my_g)

    else: 

        my_g = create_new_group(counter())

        my_groups.append(my_g)

    
my_members = {k.id:set() for k in my_groups}     # to index members and prevent dupes

for i in my_groups:
    display(HTML('<h1>GROUP RESOURCE AS YAML</h1>'))
    display(HTML(f'<pre style="color: blue; background-color: AliceBlue;">{to_yaml(i)}</pre>'))
print(f'len(my_bundle.entry)={len([x for x in my_bundle.entry if x.resource.resource_type == "Encounter"])}' )
print(f'len(my_groups)={len(my_groups)}' )

len(my_bundle.entry)=95
len(my_groups)=3


### Add the Group Members from Bundle for Each ManagingEntity

- Fetch Bundle from local dir
- get Patient Ids for each organization
  - note there are no managing Org on these patients. so for this data set we impute the organization from the the encounter to tell us which organization the patient list go with.
   (This is an issue that is system dependent only they know)
   cycle through all encounters to get all patients for each ME
- Create QR for each Member based on Q and add extension
    Use template for now


In [12]:
def make_qr(py_enc, p_id):  # make and bundle up QRs 
    now = f'{datetime.now().isoformat()}Z'
    my_qr = qr.QuestionnaireResponse(FHIR_templates.qr_template) # instantiate QR from premade template
    my_qr.id = f'{q_id}-qr-{p_id}'
    my_qr.subject.reference = f'Patient/{get_id(pyfhir.subject)}'
    my_qr.subject.display = pyfhir.subject.display   
    my_qr.authored = fhirdate.FHIRDate(now)
    #get source data using FHIR RESTful Searches for items 0 and 1
    r_dict = fetch_r('Patient',get_id(pyfhir.subject)) #get p as dict
    py_patient = patient.Patient(r_dict)
    my_qr.item[0].answer[0].valueString = to_json(py_patient.birthDate) 
    my_qr.item[1].answer[0].valueString = py_patient.gender                                                        
    my_qr.item[2].answer[0].valueString = py_enc.period.start.as_json()
    my_qr.item[3].answer[0].valueString = py_enc.id
    my_qr.item[4].answer[0].valueReference.reference = f'Encounter/{py_enc.id}'
    my_qr.item[5].answer[0].valueUrl = f'http://example.org/data/{py_enc.id}'
    
    #print(to_yaml(my_qr))
    my_qrs.append(my_qr)

def get_qr_ext(py_enc):
    p_id = get_id(pyfhir.subject)
    if pl_use_case == "all":  # only need to do this once ASSUMES START WITH ALL
        make_qr(py_enc, p_id)
    
    qr_ref = dict(
    reference = f'QuestionnaireResponse/{q_id}-qr-{p_id}',
    display = f'Argo PL List Test QuestionnaireResponse {p_id}',
     )
    
    my_qr_extension = dict(
     url = qr_ext_url,
     valueReference = qr_ref,
    )
    #print(to_yaml(my_qr_extension))
    return my_qr_extension
    
#print(f'len(my_bundle.entry)={len(my_bundle.entry)}' )
#print(my_members)
my_qrs = []

for count, i in enumerate(my_bundle.entry):
    #print(count)
    my_chars = []
    pyfhir = i.resource
    if pyfhir.resource_type == "Encounter":
        #print(count,':',pyfhir.resource_type ,i.fullUrl, i.request ) 
        sp_id = get_id(pyfhir.serviceProvider)
        
        my_chars.append(get_id(pyfhir.participant[0].individual))
      
        my_chars.append(get_id(pyfhir.location[0].location))
        print(count, my_chars)
        my_ext = get_qr_ext(pyfhir)
        my_su = dict(
            extension = [my_ext],
            reference = f"Patient/{get_id(pyfhir.subject)}",
            display = f"{pyfhir.subject.display}",
            )
        my_member = group.GroupMember(
            dict(
                entity = my_su,
                period = {'start': str(date.today())},
                inactive = False,
                )
            )
        # iterate through the groups an append to Group with ME == ServiceProvider
        # and characteristice = characterestiic  maintaining a set for each group
        for g in my_groups: # assuming a single characteristic for now..
            me_id =  get_id(g.managingEntity)
            #print(g.characteristic[0].valueReference)
            if pl_use_case == "all" or get_id(g.characteristic[0].valueReference) in my_chars:
                #print(f'my_member.entity.reference ={my_member.entity.reference}')
                if me_id == sp_id and my_member.entity.reference not in my_members[g.id]:
                    try:
                        g.member.append(my_member)
                    except AttributeError:
                        g.member=[my_member]
                    my_members[g.id].add(my_member.entity.reference)
               
for i in my_qrs:
    display(HTML('<h1>QR RESOURCE AS YAML</h1>'))
    display(HTML(f'<pre style="color: blue; background-color: AliceBlue;">{to_yaml(i)}</pre>'))
    display(HTML('...validating...&#9203;'))
    r = validate(i)
    display(HTML(f'<h2>Validation output</h1><h2>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}<hr><hr>'))
    
for i in my_groups:
    display(HTML('<h1>GROUP RESOURCE AS YAML</h1>'))
    display(HTML(f'<pre style="color: blue; background-color: AliceBlue;">{to_yaml(i)}</pre>'))
    display(HTML('...validating...&#9203;'))
    r = validate(i)
    display(HTML(f'<h2>Validation output</h1><h2>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]} <hr><hr>'))

466 ['0000016f-64f5-9833-0000-0000000000c8', '004a2230-c8b8-4b78-adc7-d37a44103a7f']


467 ['0000016f-64f5-9833-0000-000000000096', '3806554c-0e8d-455b-b666-57511f0632d8']


468 ['0000016f-64f5-9833-0000-000000000096', 'bd7875ed-e7f7-4239-a6ec-61c4fed53751']


469 ['0000016f-64f5-9833-0000-0000000001a4', '004a2230-c8b8-4b78-adc7-d37a44103a7f']


470 ['0000016f-64f5-9833-0000-0000000001a4', 'bd7875ed-e7f7-4239-a6ec-61c4fed53751']


...

WARNING,"[QuestionnaireResponse, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[QuestionnaireResponse.meta.tag[0], Line 7, Col 8]",A code with no system has no defined meaning. A system should be provided
WARNING,"[QuestionnaireResponse.meta.tag[1], Line 10, Col 8]",A code with no system has no defined meaning. A system should be provided
ERROR,"[QuestionnaireResponse, Line 71, Col 6]",LinkId 'rel-encounter-uri' not found in questionnaire
ERROR,"[QuestionnaireResponse.item[0], Line 41, Col 6]","If text exists, it must match the questionnaire definition for linkId admit-date"
ERROR,"[QuestionnaireResponse.item[0], Line 50, Col 6]","If text exists, it must match the questionnaire definition for linkId rel-encounter-id"


WARNING,"[QuestionnaireResponse, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[QuestionnaireResponse.meta.tag[0], Line 7, Col 8]",A code with no system has no defined meaning. A system should be provided
WARNING,"[QuestionnaireResponse.meta.tag[1], Line 10, Col 8]",A code with no system has no defined meaning. A system should be provided
ERROR,"[QuestionnaireResponse, Line 71, Col 6]",LinkId 'rel-encounter-uri' not found in questionnaire
ERROR,"[QuestionnaireResponse.item[0], Line 41, Col 6]","If text exists, it must match the questionnaire definition for linkId admit-date"
ERROR,"[QuestionnaireResponse.item[0], Line 50, Col 6]","If text exists, it must match the questionnaire definition for linkId rel-encounter-id"


WARNING,"[Group, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Group.meta.tag[0], Line 8, Col 8]",A code with no system has no defined meaning. A system should be provided
INFORMATION,"[Group.extension[0], Line 14, Col 6]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistsquestionnaire
INFORMATION,"[Group.member[0].entity, Line 39, Col 23]",Details for Patient/4e018248-ed8b-4111-90f2-6584d3e78ba3 matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
INFORMATION,"[Group.member[0].entity.extension[0], Line 40, Col 12]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistquestionnaireresponse
INFORMATION,"[Group.member[1].entity, Line 58, Col 23]",Details for Patient/744fe93a-7a0d-41d8-8e2d-7e1ce1e9a85b matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
INFORMATION,"[Group.member[1].entity.extension[0], Line 59, Col 12]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistquestionnaireresponse
INFORMATION,"[Group.member[2].entity, Line 77, Col 23]",Details for Patient/64fa51b4-6a38-4bd1-afef-f762f99a047a matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
INFORMATION,"[Group.member[2].entity.extension[0], Line 78, Col 12]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistquestionnaireresponse
INFORMATION,"[Group.member[3].entity, Line 96, Col 23]",Details for Patient/c9fe923e-cc0c-4986-a799-911fa95576dc matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
INFORMATION,"[Group.member[3].entity.extension[0], Line 97, Col 12]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistquestionnaireresponse


WARNING,"[Group, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Group.meta.tag[0], Line 8, Col 8]",A code with no system has no defined meaning. A system should be provided
INFORMATION,"[Group.extension[0], Line 14, Col 6]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistsquestionnaire
INFORMATION,"[Group.member[0].entity, Line 39, Col 23]",Details for Patient/b26fbdf0-9b79-4807-a380-f84f0ae83aed matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
INFORMATION,"[Group.member[0].entity.extension[0], Line 40, Col 12]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistquestionnaireresponse
INFORMATION,"[Group.member[1].entity, Line 58, Col 23]",Details for Patient/ace1d67a-3083-4f37-a5d6-b5dbea300a93 matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
INFORMATION,"[Group.member[1].entity.extension[0], Line 59, Col 12]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistquestionnaireresponse
INFORMATION,"[Group.member[2].entity, Line 77, Col 23]",Details for Patient/825b35dd-49be-4b46-98ce-9f66083b1c35 matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
INFORMATION,"[Group.member[2].entity.extension[0], Line 78, Col 12]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistquestionnaireresponse
INFORMATION,"[Group.member[3].entity, Line 96, Col 23]",Details for Patient/938c1cca-edb4-464f-93f7-5070df3d87fa matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
INFORMATION,"[Group.member[3].entity.extension[0], Line 97, Col 12]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistquestionnaireresponse


WARNING,"[Group, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Group.meta.tag[0], Line 8, Col 8]",A code with no system has no defined meaning. A system should be provided
INFORMATION,"[Group.extension[0], Line 14, Col 6]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistsquestionnaire
INFORMATION,"[Group.member[0].entity, Line 39, Col 23]",Details for Patient/5812ffab-0650-47e1-ae6c-c32d4d0b330d matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
INFORMATION,"[Group.member[0].entity.extension[0], Line 40, Col 12]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistquestionnaireresponse
INFORMATION,"[Group.member[1].entity, Line 58, Col 23]",Details for Patient/8b74fb11-db4d-438d-90d0-a4962a3dcb02 matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
INFORMATION,"[Group.member[1].entity.extension[0], Line 59, Col 12]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistquestionnaireresponse
INFORMATION,"[Group.member[2].entity, Line 77, Col 23]",Details for Patient/f422d5a4-a267-4d69-9cd2-4f0e8935951b matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
INFORMATION,"[Group.member[2].entity.extension[0], Line 78, Col 12]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistquestionnaireresponse
INFORMATION,"[Group.member[3].entity, Line 96, Col 23]",Details for Patient/1cfc7b78-44db-4c00-8757-f8025782119c matching against Profilehttp://hl7.org/fhir/StructureDefinition/Patient
INFORMATION,"[Group.member[3].entity.extension[0], Line 97, Col 12]",Unknown extension http://registry.fhir.org/argonaut/StructureDefinition/patientlistquestionnaireresponse


### Bundle into Transaction/Validate/Save/Load to ref FHIR Server

- Create Transaction bundle using PUT to preserve ids
- put the dependent resources first 
  order is Patient, Pract, Org, Location, Q, QR, Careteam, Condition, Observation, Encounter, Group
- Use static example organization from US Core

  

In [13]:
my_b = None
display(HTML('......Bundle into transaction...... &#9203;'))
test_server = 'http://test.fhir.org/r4'
my_res = my_qrs + my_groups
for r in my_res:
    try:
        my_b = bundle_me(test_server, r, my_b)
    except NameError:
        my_b = bundle_me(test_server, r)

display(HTML('<h1>BUNDLE RESOURCE AS YAML</h1>'))
display(HTML(f'<pre style="color: blue; background-color: AliceBlue;">{to_yaml(my_b)}</pre>'))
display(HTML('...validating...&#9203;'))
r = validate(my_b)
display(HTML(f'<h2>Validation output</h1><h2>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]} <hr><hr>'))

WARNING,"[Bundle.entry[0].resource.ofType(QuestionnaireResponse), Line 25, Col 21]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Bundle.entry[0].resource.ofType(QuestionnaireResponse).meta.tag[0], Line 15, Col 14]",A code with no system has no defined meaning. A system should be provided
WARNING,"[Bundle.entry[0].resource.ofType(QuestionnaireResponse).meta.tag[1], Line 18, Col 14]",A code with no system has no defined meaning. A system should be provided
ERROR,"[Bundle.entry[0].resource.ofType(QuestionnaireResponse), Line 79, Col 12]",LinkId 'rel-encounter-uri' not found in questionnaire
ERROR,"[Bundle.entry[0].resource.ofType(QuestionnaireResponse).item[0], Line 49, Col 12]","If text exists, it must match the questionnaire definition for linkId admit-date"
ERROR,"[Bundle.entry[0].resource.ofType(QuestionnaireResponse).item[0], Line 58, Col 12]","If text exists, it must match the questionnaire definition for linkId rel-encounter-id"
WARNING,"[Bundle.entry[1].resource.ofType(QuestionnaireResponse), Line 113, Col 21]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Bundle.entry[1].resource.ofType(QuestionnaireResponse).meta.tag[0], Line 103, Col 14]",A code with no system has no defined meaning. A system should be provided
WARNING,"[Bundle.entry[1].resource.ofType(QuestionnaireResponse).meta.tag[1], Line 106, Col 14]",A code with no system has no defined meaning. A system should be provided
ERROR,"[Bundle.entry[1].resource.ofType(QuestionnaireResponse), Line 167, Col 12]",LinkId 'rel-encounter-uri' not found in questionnaire
ERROR,"[Bundle.entry[1].resource.ofType(QuestionnaireResponse).item[0], Line 137, Col 12]","If text exists, it must match the questionnaire definition for linkId admit-date"


### Write to File

In [14]:
if not console_only:
    file_name = f'argo-pl-{pl_use_case}-{my_b.id}.json'

    my_path = Path.cwd()/ 'test_out' / file_name
    display(HTML(f'&#9203...writing bundle resource as json file  to {my_path}... '))
    my_path.write_text(to_json(my_b))


    display(HTML(f'&#8987; Done!!!'))

### Post to Test Server

In [15]:
if not console_only:
    r = load_transaction(my_b)
    try:
        display(HTML(
            '<h1>Post Response</h1>'
            f'<h3>Status Code = {r.status_code}</h3><br />'
            f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
            f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
            '===============================================<br /><br /><br />'
            ))
    except KeyError:
        display(HTML(
            '<h1>Post Response</h1>'
            f'<h3>Status Code = {r.status_code}</h3><br />'
            f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
            #f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
            '===============================================<br /><br /><br />'
            ))   